## Optimizing an Apache MXNet model for AWS DeepLens

This notebook shows you how to use the Intel Deep Learning Deployment Toolkit to optimize an MXNet model for Deep Lens.

In [ ]:
%env TOOLKIT_BUCKET=s3://jsimon-public-us/
%env TOOLKIT_NAME=toolkit.tgz
%env TOOLKIT_DIR=l_deeplearning_deploymenttoolkit_2017.1.0.5852

%env MODEL_BUCKET=s3://jsimon-public-us/
%env MODEL_NAME=Inception-BN

%env OPT_DIR=/opt/intel/deeplearning_deploymenttoolkit/deployment_tools/model_optimizer/model_optimizer_mxnet
%env OPT_PRECISION=FP16
%env OPT_FUSE=YES

In [ ]:
%%bash

echo "*** Downloading toolkit"
aws s3 cp $TOOLKIT_BUCKET$TOOLKIT_NAME .
echo "*** Installing toolkit"
tar xfz $TOOLKIT_NAME
cd $TOOLKIT_DIR
chmod 755 install.sh
sudo ./install.sh -s silent.cfg 
echo "*** Done"

In [ ]:
%%bash

#conda create -n intel_toolkit -y
python -m ipykernel install --user --name intel_toolkit --display-name "intel_toolkit"

source activate intel_toolkit
cd $OPT_DIR
pip install -r requirements.txt 

In [ ]:
%%bash

echo "*** Downloading model"
aws s3 cp $MODEL_BUCKET$MODEL_NAME"-symbol.json" .
aws s3 cp $MODEL_BUCKET$MODEL_NAME"-0000.params" .
echo "*** Done"

In [ ]:
%%bash

source activate intel_toolkit
echo "*** Converting model"
python $OPT_DIR/mo_mxnet_converter.py --models-dir . --output-dir . --model-name $MODEL_NAME --precision $OPT_PRECISION --fuse $OPT_FUSE
ls mxnet_$MODEL_NAME*
echo "*** Done"